In [0]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from IPython.display import clear_output
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


In [2]:
# Importamos el csv
df = pd.read_csv('files.csv').drop(columns="Unnamed: 0")
client = df['Files'].values
df.head(5)

,Files
0,wp-admin/js/post.js
1,wp-includes/css/media-views-rtl.css
2,wp-includes/js/plupload/plupload.html5.js
3,wp-includes/js/wp-lists.js
4,wp-includes/css/wp-pointer-rtl.css


In [3]:
df = pd.DataFrame(columns=client) 
df

,wp-admin/js/post.js,wp-includes/css/media-views-rtl.css,wp-includes/js/plupload/plupload.html5.js,wp-includes/js/wp-lists.js,wp-includes/css/wp-pointer-rtl.css,wp-includes/js/plupload/plupload.html4.js,wp-includes/css/editor.css,wp-includes/js/plupload/plupload.flash.js,wp-includes/js/tinymce/wp-tinymce.js.gz,wp-includes/js/imgareaselect/jquery.imgareaselect.js,wp-includes/css/dashicons.css,wp-includes/css/buttons-rtl.css,wp-admin/js/common.dev.js,wp-admin/js/link.js,wp-includes/js/imgareaselect/jquery.imgareaselect.dev.js,wp-admin/js/customize-controls.dev.js,wp-admin/css/install.css,wp-includes/css/admin-bar.css,wp-includes/js/plupload/wp-plupload.dev.js,wp-admin/js/nav-menu.js,wp-includes/js/tinymce/plugins/wordpress/editor_plugin_src.js,wp-includes/css/media-views.css,wp-includes/js/jquery/jquery.js,wp-includes/js/tinymce/plugins/wpeditimage/editor_plugin_src.js,wp-includes/js/customize-preview.js,wp-includes/js/plupload/plupload.silverlight.js,wp-admin/css/install-rtl.css,wp-admin/css/edit.css,wp-admin/js/dashboard.js,wp-includes/css/wp-pointer.css,wp-includes/css/admin-bar-rtl.css,wp-admin/js/post.dev.js,wp-includes/js/wp-lists.dev.js,wp-admin/css/wp-admin-rtl.css,wp-admin/css/wp-admin.dev.css,wp-includes/js/swfobject.js,wp-includes/js/plupload/wp-plupload.js,wp-admin/js/theme.js,wp-includes/js/plupload/plupload.js,wp-includes/js/plupload/handlers.dev.js,wp-admin/js/customize-controls.js,wp-includes/css/editor.dev.css,wp-includes/css/buttons.css,wp-admin/js/link.dev.js,wp-includes/js/customize-preview.dev.js,Version


In [0]:
import concurrent.futures
import requests
import threading
import time
from multiprocessing import Pool
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from termcolor import colored 


'''
The operating system is in control of when your task gets interrupted and another task starts, 
any data that is shared between the threads needs to be protected, or thread-safe. 
Unfortunately requests.Session() is not thread-safe.
Exists something called thread local storage. 
Threading.local() creates an object that look like a global but is specific to each individual thread.
# https://realpython.com/python-concurrency/
'''
threadLocal = threading.local()


def get_session():
    if not hasattr(threadLocal, "session"):
        threadLocal.session = requests.Session()
    return threadLocal.session


def get_wp_version(url):
    headers = {
        'X-Api-Key': 'wappalyzer.api.demo.key',
    }
    try:
        response = requests.get('https://api.wappalyzer.com/lookup/v1/?url='+url, headers=headers).json()
        for r in response:
            for k, v in r.items():
                if k == "applications":
                    for aplication in v:
                        if "WordPress" in aplication.get("name", ):
                            name = aplication.get("name", )
                            version = aplication.get("versions", )

        version = ''.join(version)
        return version
    except:
        return "Null"
    
    
def get_wp_manual_version(url):
    session = get_session()
    try:
        with session.get(url, verify=False, timeout=5) as response:
            if response.status_code == 200:
                try:
                    soup = BeautifulSoup(response.content, 'html.parser')
                    version = soup.find('meta', content=re.compile("WordPress\s\d"))
                    version = version.get('content').split(" ")[1]
                    return version
                except:
                    return "Null"
            else:
                return "Null"
    except:
        return "Null"
    

def get_bytes(url):
    session = get_session()
    try:
        with session.get(url, verify=False, timeout=5) as response:
            if response.status_code == 200:
                bytess = len(response.content)
            elif response.status_code != 200:
                bytess = 0
    except:
        bytess = 0
    return bytess


def create_dataset_to_predict(url):
    main_array = []
    bytess = []
    time_scan = time.time()
    session = get_session()
    version = get_wp_manual_version(url)
    urlypath = []
    for c in client:  
        if "Version" in c:
            continue
        urlypath.append(url+c)
        
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        bytess = list(executor.map(get_bytes, urlypath))
        for b in bytess:
            main_array.append(b)  
                
    main_array.append(version) 
    if len(main_array) == 46:
        try:
            df.loc[len(df)] = main_array
            return df
        except:
            print(url)
 
def create_dataset(url):
    #clear_output()
    main_array = []
    global count_iter
    
    if not count_iter%100:
        clear_output()
    count_iter += 1
    
    bytess = []
    time_scan = time.time()
    session = get_session()
    version = get_wp_manual_version(url)
    # Si no encuentra la versión ya descartamos la url y añadimos un array de ceros.
    if version != "Null":
        urlypath = []
        for c in client:  
            if "Version" in c:
                continue
            urlypath.append(url+c)
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
            bytess = list(executor.map(get_bytes, urlypath))
            for b in bytess:
                main_array.append(b)            
 
        print(colored("["+str(count_iter)+"] "+url+" | WP-Version: "+version+" ("+str(round(float(0.00000095367432)*sum(bytess), 3))+"/Kb) "+ f' | Tiempo Escaneo: {round(time.time() - time_scan, 3)}', "green", attrs=['bold']))
    else:
        listofzeros = [0] * 45
        for z in listofzeros:
            main_array.append(z)
        print(colored("["+str(count_iter)+"] "+url+" | WP-Version: "+version+" ("+str(round(float(0.00000095367432)*sum(bytess), 3))+"/Kb) "+ f' | Tiempo Escaneo: {round(time.time() - time_scan, 3)}', "red"))
    
    main_array.append(version) 
    if len(main_array) == 46:
        try:
            df.loc[len(df)] = main_array
            return df
        except:
            print(url)



def create_dataset_multiple(urls):
    global count_iter 
    count_iter = -12
    with concurrent.futures.ThreadPoolExecutor(max_workers=12) as executor:
        df = pd.concat(executor.map(create_dataset, urls))
        # Al concatenar perderemos el index lo reseteamos
        #df.reset_index()
    return df


def get_array(file):
    with open(file, "r") as f:
        urls = []
        for line in f:
            urls.append(line.replace("\n", "")) 
    return urls

In [0]:
# Haremos peticiones a las URLs con multiprocesado
# https://blog.floydhub.com/multiprocessing-vs-threading-in-python-what-every-data-scientist-needs-to-know/
# https://realpython.com/python-concurrency/

# Pasamos unas determinadas URLs a una lista
df = pd.read_csv('final.csv')
# Get first 200 
#urls = df['URLs'].head(200)

# Random urls, sample function gets by default 1 row random
urls = df['URLs'].sample(n=100000) 


# Creamos un dataframe en blanco
df = pd.DataFrame(columns=client)

start = time.time()
print("Creating dataset:")
print("Checking URLs:")
create_dataset_multiple(urls)
print(f'Tiempo ejecucion: {time.time() - start}')

#df

[27101] https://travelsofadam.com/ | WP-Version: 5.2.3 (0.974/Kb)  | Tiempo Escaneo: 2.111[27101] http://santeria.mamaine.com/ | WP-Version: 4.9.11 (1.567/Kb)  | Tiempo Escaneo: 7.122

[27104] https://hillconsulting.nl/ | WP-Version: 5.2.3 (0.974/Kb)  | Tiempo Escaneo: 17.117
[27105] http://postandnickel.com/ | WP-Version: 4.9.11 (1.163/Kb)  | Tiempo Escaneo: 5.679
[27106] http://www.mobiveil.com/ | WP-Version: 4.1.27 (0.767/Kb)  | Tiempo Escaneo: 6.311
[27107] https://theblissfulmind.com/ | WP-Version: 5.2.3 (0.092/Kb)  | Tiempo Escaneo: 0.838
[27108] http://primarycomputing.co.uk/ | WP-Version: Null (0.0/Kb)  | Tiempo Escaneo: 0.175
[27108] https://www.lawnet.gov.lk/ | WP-Version: 4.8.1 (0.989/Kb)  | Tiempo Escaneo: 11.933
[27110] http://vetekatten.se/ | WP-Version: Null (0.0/Kb)  | Tiempo Escaneo: 1.222
[27111] https://www.alonti.com/ | WP-Version: Null (0.0/Kb)  | Tiempo Escaneo: 0.173
[27112] https://www.celeb.vn/ | WP-Version: 5.2.3 (0.974/Kb)  | Tiempo Escaneo: 8.179
[27113] htt

In [0]:
df = df[df.Version != 'Null']
df.to_csv('20000.csv')
df

,wp-admin/js/post.js,wp-includes/css/media-views-rtl.css,wp-includes/js/plupload/plupload.html5.js,wp-includes/js/wp-lists.js,wp-includes/css/wp-pointer-rtl.css,wp-includes/js/plupload/plupload.html4.js,wp-includes/css/editor.css,wp-includes/js/plupload/plupload.flash.js,wp-includes/js/tinymce/wp-tinymce.js.gz,wp-includes/js/imgareaselect/jquery.imgareaselect.js,wp-includes/css/dashicons.css,wp-includes/css/buttons-rtl.css,wp-admin/js/common.dev.js,wp-admin/js/link.js,wp-includes/js/imgareaselect/jquery.imgareaselect.dev.js,wp-admin/js/customize-controls.dev.js,wp-admin/css/install.css,wp-includes/css/admin-bar.css,wp-includes/js/plupload/wp-plupload.dev.js,wp-admin/js/nav-menu.js,wp-includes/js/tinymce/plugins/wordpress/editor_plugin_src.js,wp-includes/css/media-views.css,wp-includes/js/jquery/jquery.js,wp-includes/js/tinymce/plugins/wpeditimage/editor_plugin_src.js,wp-includes/js/customize-preview.js,wp-includes/js/plupload/plupload.silverlight.js,wp-admin/css/install-rtl.css,wp-admin/css/edit.css,wp-admin/js/dashboard.js,wp-includes/css/wp-pointer.css,wp-includes/css/admin-bar-rtl.css,wp-admin/js/post.dev.js,wp-includes/js/wp-lists.dev.js,wp-admin/css/wp-admin-rtl.css,wp-admin/css/wp-admin.dev.css,wp-includes/js/swfobject.js,wp-includes/js/plupload/wp-plupload.js,wp-admin/js/theme.js,wp-includes/js/plupload/plupload.js,wp-includes/js/plupload/handlers.dev.js,wp-admin/js/customize-controls.js,wp-includes/css/editor.dev.css,wp-includes/css/buttons.css,wp-admin/js/link.dev.js,wp-includes/js/customize-preview.dev.js,Version
4,37407,49356,0,25282,4027,0,32907,0,0,38132,50341,9868,0,3875,0,0,5911,25857,0,42482,0,49358,96873,0,27969,0,5912,27635,18858,4022,25862,0,0,455,0,10231,12806,54392,60314,0,290802,0,9867,0,0,5.2.3
5,37407,49356,52044,25282,4027,52044,32907,52044,52044,38132,50341,9868,52044,3875,52044,52044,5911,25857,52044,42482,52044,49358,96873,52044,27969,52044,5912,52044,18858,4022,25862,52044,52044,455,52044,10231,12806,54392,60314,52044,290802,52044,9867,52044,52044,5.2.3
6,37407,49356,0,25282,4027,0,32907,0,0,38132,50341,9868,0,3875,0,0,5911,25857,0,42482,0,49358,96873,0,27969,0,5912,27635,18858,4022,25862,0,0,455,0,10231,12806,54392,60314,0,290802,0,9867,0,0,5.2.3
7,37355,48382,0,25282,4024,0,32823,0,0,38132,48809,9599,0,3875,0,0,6982,25178,0,42475,0,48383,96873,0,27969,0,6983,28996,19605,4019,25183,0,0,421,0,10231,12806,54392,60314,0,290802,0,9598,0,0,5.1.2
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.9.10
9,37407,49356,0,25282,4027,0,32907,0,0,38132,50341,9868,0,3875,0,0,5911,25857,0,42482,0,49358,96873,0,27969,0,5912,27635,18858,4022,25862,0,0,455,0,10231,12806,54392,60314,0,290802,0,9867,0,0,5.2.3
10,37407,49356,0,25282,4027,0,32907,0,0,38132,50341,9868,0,3875,0,0,5911,25857,0,42482,0,49358,96873,0,27969,0,5912,27635,18858,4022,25862,0,0,455,0,10231,12806,54392,60314,0,290802,0,9867,0,0,5.2.3
11,37407,49356,0,25282,4027,0,32907,0,0,38132,50341,9868,0,3875,0,0,5911,25857,0,42482,0,49358,96873,0,27969,0,5912,27635,18858,4022,25862,0,0,455,0,10231,12806,54392,60314,0,290802,0,9867,0,0,5.2.3
12,37407,49356,0,25282,4027,0,32907,0,0,38132,50341,9868,0,3875,0,0,5911,25857,0,42482,0,49358,96873,0,27969,0,5912,27635,18858,4022,25862,0,0,455,0,10231,12806,54392,60314,0,290802,0,9867,0,0,5.2.3
13,37308,48995,68122,25282,4027,68122,32907,68122,68120,38132,50341,9702,68105,3875,68137,68117,6989,25781,68123,42475,68144,48996,96873,68146,27969,68128,6990,68104,18858,4022,25786,68103,68110,455,68109,10231,12806,54392,60314,68120,290802,68110,9701,68103,68119,5.2.2


In [0]:
# Creamos un dataframe en blanco
df = pd.DataFrame(columns=client)

URLS_test = ["https://emetel.net/", "https://www.be-sec.net/", "https://wordpress.org/"]
create_dataset_multiple(URLS_test)
df

[-9] https://wordpress.org/ | WP-Version: Null (0.0/Kb)  | Tiempo Escaneo: 0.128
[-9] https://www.be-sec.net/ | WP-Version: 5.0.6 (1.162/Kb)  | Tiempo Escaneo: 7.246
[-9] https://emetel.net/ | WP-Version: 4.9.11 (1.163/Kb)  | Tiempo Escaneo: 13.116


,wp-admin/js/post.js,wp-includes/css/media-views-rtl.css,wp-includes/js/plupload/plupload.html5.js,wp-includes/js/wp-lists.js,wp-includes/css/wp-pointer-rtl.css,wp-includes/js/plupload/plupload.html4.js,wp-includes/css/editor.css,wp-includes/js/plupload/plupload.flash.js,wp-includes/js/tinymce/wp-tinymce.js.gz,wp-includes/js/imgareaselect/jquery.imgareaselect.js,wp-includes/css/dashicons.css,wp-includes/css/buttons-rtl.css,wp-admin/js/common.dev.js,wp-admin/js/link.js,wp-includes/js/imgareaselect/jquery.imgareaselect.dev.js,wp-admin/js/customize-controls.dev.js,wp-admin/css/install.css,wp-includes/css/admin-bar.css,wp-includes/js/plupload/wp-plupload.dev.js,wp-admin/js/nav-menu.js,wp-includes/js/tinymce/plugins/wordpress/editor_plugin_src.js,wp-includes/css/media-views.css,wp-includes/js/jquery/jquery.js,wp-includes/js/tinymce/plugins/wpeditimage/editor_plugin_src.js,wp-includes/js/customize-preview.js,wp-includes/js/plupload/plupload.silverlight.js,wp-admin/css/install-rtl.css,wp-admin/css/edit.css,wp-admin/js/dashboard.js,wp-includes/css/wp-pointer.css,wp-includes/css/admin-bar-rtl.css,wp-admin/js/post.dev.js,wp-includes/js/wp-lists.dev.js,wp-admin/css/wp-admin-rtl.css,wp-admin/css/wp-admin.dev.css,wp-includes/js/swfobject.js,wp-includes/js/plupload/wp-plupload.js,wp-admin/js/theme.js,wp-includes/js/plupload/plupload.js,wp-includes/js/plupload/handlers.dev.js,wp-admin/js/customize-controls.js,wp-includes/css/editor.dev.css,wp-includes/css/buttons.css,wp-admin/js/link.dev.js,wp-includes/js/customize-preview.dev.js,Version
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Null
1,37410,47837,0,25235,4026,0,32204,0,208317,38132,48813,9410,0,2247,0,0,7020,25094,0,42256,0,47838,96874,0,27836,0,7021,28637,16585,4021,25099,0,0,421,0,10231,12726,54364,60314,0,289044,0,9409,0,0,5.0.6
2,37410,48049,0,25235,4026,0,32033,0,208202,38132,48813,9484,0,2247,0,0,7020,25094,0,42256,0,48051,96874,0,27836,0,7021,28901,16585,4021,25099,0,0,421,0,10231,12726,54364,60314,0,289044,0,9483,0,0,4.9.11


In [0]:
# Creamos un dataframe en blanco
df = pd.DataFrame(columns=client)

create_dataset_to_predict("https://wordpress.org/")
df.to_csv("wordpress.csv")